In [35]:
#########
### importing bla bla bla ####
import pandas as pd
import numpy as np
import seaborn as sb
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)

from sklearn.cross_validation import train_test_split
from pandas import Series, DataFrame
from pylab import rcParams


from sklearn.metrics import classification_report,accuracy_score
sb.set(style="white")
sb.set(style="whitegrid", color_codes=True)
import itertools
from sklearn import tree
import graphviz
from sklearn.model_selection import GridSearchCV,StratifiedKFold, LeaveOneOut
from sklearn.datasets import make_regression
from sklearn.ensemble import AdaBoostRegressor

import warnings; warnings.simplefilter('ignore')






### done with the imports ###



In [36]:
### reading data and printing the data ###
data = pd.read_csv('classical_feature.csv',names=['ID','TruncatedID','Mean','SD','AUC','Peak','MaxIndex','Halfpk','Skew','HHMean','Fat','Carb','Protein'
])
# data = data.dropna()
print(data.shape)
print(list(data.columns))


### done with reading data and printing the data ###

(171, 13)
['ID', 'TruncatedID', 'Mean', 'SD', 'AUC', 'Peak', 'MaxIndex', 'Halfpk', 'Skew', 'HHMean', 'Fat', 'Carb', 'Protein']


In [37]:
### CHO
CHO_data = data.drop(['ID','Protein','Fat'], axis=1) #taking only the Carb target column
print(CHO_data.Carb.unique())# Finding the unique values in that column
print(CHO_data.TruncatedID.unique()) #finiding unique IDs


[51 48 54]
['2133-002' '2133-022' '2133-020' '2133-004' '2133-010' '2133-015'
 '2133-040' '2133-013' '2133-008' '2133-019' '2133-001' '2133-024'
 '2133-011' '2133-018' '2133-006' '2133-032' '2133-025' '2133-017'
 '2133-028' '2133-039' '2133-021' '2133-030' '2133-033' '2133-037'
 '2133-026' '2133-012' '2133-036' '2133-035' '2133-041' '2133-009']


In [38]:
###for LOW-HIGH

CHO_LH_data = CHO_data[CHO_data['Carb'].isin([48,54])] #considering the two classes for binary classification
CHO_LH_data.loc[CHO_LH_data.Carb == 48, 'CHO_bin'] = 0 #giving them binary values, as double variables sometimes create error
CHO_LH_data.loc[CHO_LH_data.Carb== 54, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Carb'], axis=1)
#the above 4 lines will be different for different classification, the remaining code will be same
X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin'] #Feature set
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin'] #target value/label

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009'] #needed for manual loso
final_accuracy =[] #will be used for finding mean across nested cross validation
for i in range(0,30): #as we have 30 IDs
    print("For fold :", i+1) #will print the current fold number, this is for initial check, can delete it
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])] #test data
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]#train data
    X_train = train.loc[:, train.columns != 'CHO_bin'] #test feature
    Y_train = train.loc[:, train.columns == 'CHO_bin'] #test label
    X_test = test.loc[:, test.columns != 'CHO_bin']#train feature
    
    Y_test =test.loc[:, test.columns == 'CHO_bin']#train label
    train_ids = X_train['TruncatedID']#needed for inner cross-validation
  
    train_ids = np.array(train_ids)#needed for inner cross-validation
    train_ids = np.unique(train_ids)#needed for inner cross-validation
#     print(train_ids[1])
    X_train = X_train.drop(['TruncatedID'],axis = 1) #dropping this column as it is not a feature for classification
    X_test = X_test.drop(['TruncatedID'],axis = 1)#dropping this column as it is not a feature for classification
    max_acc = 0 #initializing maximum accuracy which will be needed later
    for k in range(1,6): #ranges of decision tree depth
        ypred_all = [] 
        for j in range(0,29): #as inner cross validation ha 29 IDs
            rng = np.random.RandomState(1)
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])] #validation set
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]#inner train set
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin'] #inner train set features
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']# inner train set label
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']#validation set features
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']#validation set labels
            xtr = xtr.drop(['TruncatedID'],axis = 1)#We dont need this column for classification 
            xvl = xvl.drop(['TruncatedID'],axis = 1)#We dont need this column for classification
            rf1 = tree.DecisionTreeClassifier(max_depth=k)#Decision tree
            rf = AdaBoostRegressor(rf1)
            rf = rf.fit(xtr,ytr)
            ypred = rf.predict(xvl)#prediction
            ypred_all.append(accuracy_score(yvl,ypred))
#         ypred_all = np.array(ypred_all)
        ypred_mean = np.mean(ypred_all)#finding mean of all accuracy
#         print(k,ypred_mean)
        if max_acc<ypred_mean:#finding the depth which gives maximum accuracy
            max_acc = ypred_mean
            best_param = k
#     print(max_acc, best_param)
    rf1 = tree.DecisionTreeClassifier(max_depth=best_param) #using the hyperparameter from inner cross-validation to build decision tree for outer cross-validation
    rf = AdaBoostRegressor(rf1) 
    rf = rf.fit(X_train,Y_train)
    Y_pred = rf.predict(X_test)
#     print(classification_report(Y_test,Y_pred))
#     print('Accuracy for test set :',accuracy_score(Y_test,Y_pred))
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
print("The final accuracy is : ", np.mean(final_accuracy))#final result (mean of accuracies in outer CV)
            
    

For fold : 1
For fold : 2
For fold : 3
For fold : 4
For fold : 5
For fold : 6
For fold : 7
For fold : 8
For fold : 9
For fold : 10
For fold : 11
For fold : 12
For fold : 13
For fold : 14
For fold : 15
For fold : 16
For fold : 17
For fold : 18
For fold : 19
For fold : 20
For fold : 21
For fold : 22
For fold : 23
For fold : 24
For fold : 25
For fold : 26
For fold : 27
For fold : 28
For fold : 29
For fold : 30
The final accuracy is :  0.6361111111111112


In [39]:
###for Medium-HIGH

CHO_LH_data = CHO_data[CHO_data['Carb'].isin([51,54])]
CHO_LH_data.loc[CHO_LH_data.Carb == 51, 'CHO_bin'] = 0
CHO_LH_data.loc[CHO_LH_data.Carb== 54, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Carb'], axis=1)

X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin']
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin']

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009']
final_accuracy =[]
for i in range(0,30):
    print("For fold :", i+1)
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])]
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]
    X_train = train.loc[:, train.columns != 'CHO_bin']
    Y_train = train.loc[:, train.columns == 'CHO_bin']
    X_test = test.loc[:, test.columns != 'CHO_bin']
    
    Y_test =test.loc[:, test.columns == 'CHO_bin']
    train_ids = X_train['TruncatedID']
  
    train_ids = np.array(train_ids)
    train_ids = np.unique(train_ids)
#     print(train_ids[1])
    X_train = X_train.drop(['TruncatedID'],axis = 1)
    X_test = X_test.drop(['TruncatedID'],axis = 1)
    max_acc = 0
    for k in range(1,6):
        ypred_all = []
        for j in range(0,29): #as inner cross validation ha 29 IDs
            rng = np.random.RandomState(1)
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])] #validation set
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]#inner train set
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin'] #inner train set features
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']# inner train set label
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']#validation set features
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']#validation set labels
            xtr = xtr.drop(['TruncatedID'],axis = 1)#We dont need this column for classification 
            xvl = xvl.drop(['TruncatedID'],axis = 1)#We dont need this column for classification
            rf1 = tree.DecisionTreeClassifier(max_depth=k)#Decision tree
            rf = AdaBoostRegressor(rf1)
            rf = rf.fit(xtr,ytr)
            ypred = rf.predict(xvl)#prediction
            ypred_all.append(accuracy_score(yvl,ypred))
#         ypred_all = np.array(ypred_all)
        ypred_mean = np.mean(ypred_all)#finding mean of all accuracy
#         print(k,ypred_mean)
        if max_acc<ypred_mean:#finding the depth which gives maximum accuracy
            max_acc = ypred_mean
            best_param = k
#     print(max_acc, best_param)
    rf1 = tree.DecisionTreeClassifier(max_depth=best_param) #using the hyperparameter from inner cross-validation to build decision tree for outer cross-validation
    rf = AdaBoostRegressor(rf1) 
    rf = rf.fit(X_train,Y_train)
    Y_pred = rf.predict(X_test)
#     print(classification_report(Y_test,Y_pred))
#     print('Accuracy for test set :',accuracy_score(Y_test,Y_pred))
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
print("The final accuracy is : ", np.mean(final_accuracy))#final result (mean of accuracies in outer CV)
            
    

For fold : 1
For fold : 2
For fold : 3
For fold : 4
For fold : 5
For fold : 6
For fold : 7
For fold : 8
For fold : 9
For fold : 10
For fold : 11
For fold : 12
For fold : 13
For fold : 14
For fold : 15
For fold : 16
For fold : 17
For fold : 18
For fold : 19
For fold : 20
For fold : 21
For fold : 22
For fold : 23
For fold : 24
For fold : 25
For fold : 26
For fold : 27
For fold : 28
For fold : 29
For fold : 30
The final accuracy is :  0.6944444444444444


In [40]:
###for LOW-Medium

CHO_LH_data = CHO_data[CHO_data['Carb'].isin([48,51])]
CHO_LH_data.loc[CHO_LH_data.Carb == 48, 'CHO_bin'] = 0
CHO_LH_data.loc[CHO_LH_data.Carb== 51, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Carb'], axis=1)

X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin']
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin']

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009']
final_accuracy =[]
for i in range(0,30):
    print("For fold :", i+1)
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])]
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]
    X_train = train.loc[:, train.columns != 'CHO_bin']
    Y_train = train.loc[:, train.columns == 'CHO_bin']
    X_test = test.loc[:, test.columns != 'CHO_bin']
    
    Y_test =test.loc[:, test.columns == 'CHO_bin']
    train_ids = X_train['TruncatedID']
  
    train_ids = np.array(train_ids)
    train_ids = np.unique(train_ids)
#     print(train_ids[1])
    X_train = X_train.drop(['TruncatedID'],axis = 1)
    X_test = X_test.drop(['TruncatedID'],axis = 1)
    max_acc = 0
    for k in range(1,6):
        ypred_all = []
        for j in range(0,29): #as inner cross validation ha 29 IDs
            rng = np.random.RandomState(1)
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])] #validation set
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]#inner train set
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin'] #inner train set features
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']# inner train set label
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']#validation set features
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']#validation set labels
            xtr = xtr.drop(['TruncatedID'],axis = 1)#We dont need this column for classification 
            xvl = xvl.drop(['TruncatedID'],axis = 1)#We dont need this column for classification
            rf1 = tree.DecisionTreeClassifier(max_depth=k)#Decision tree
            rf = AdaBoostRegressor(rf1)
            rf = rf.fit(xtr,ytr)
            ypred = rf.predict(xvl)#prediction
            ypred_all.append(accuracy_score(yvl,ypred))
#         ypred_all = np.array(ypred_all)
        ypred_mean = np.mean(ypred_all)#finding mean of all accuracy
#         print(k,ypred_mean)
        if max_acc<ypred_mean:#finding the depth which gives maximum accuracy
            max_acc = ypred_mean
            best_param = k
#     print(max_acc, best_param)
    rf1 = tree.DecisionTreeClassifier(max_depth=best_param) #using the hyperparameter from inner cross-validation to build decision tree for outer cross-validation
    rf = AdaBoostRegressor(rf1) 
    rf = rf.fit(X_train,Y_train)
    Y_pred = rf.predict(X_test)
#     print(classification_report(Y_test,Y_pred))
#     print('Accuracy for test set :',accuracy_score(Y_test,Y_pred))
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
print("The final accuracy is : ", np.mean(final_accuracy))#final result (mean of accuracies in outer CV)
            
    

For fold : 1
For fold : 2
For fold : 3
For fold : 4
For fold : 5
For fold : 6
For fold : 7
For fold : 8
For fold : 9
For fold : 10
For fold : 11
For fold : 12
For fold : 13
For fold : 14
For fold : 15
For fold : 16
For fold : 17
For fold : 18
For fold : 19
For fold : 20
For fold : 21
For fold : 22
For fold : 23
For fold : 24
For fold : 25
For fold : 26
For fold : 27
For fold : 28
For fold : 29
For fold : 30
The final accuracy is :  0.6611111111111111
